In [ ]:
# !pip install requests-html requests pandas

In [174]:
pip install pandas



     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ---------------------------------------- 61.2/61.2 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.7 MB ? eta -:--:--
    --------------------------------------- 0.3/10.7 MB 3.9 MB/s eta 0:00:03
   - -------------------------------------- 0.4/10.7 MB 3.7 MB/s eta 0:00:03
   - -------------------------------------- 0.5/10.7 MB 2.9 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/10.7 MB 3.0 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/10.7 MB 3.0 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/10.7 MB 3.0 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/10.7 MB 3.0 MB/s eta 0:00:04
   ---- ----------------------------------- 1.1/10.7 MB 2.9 MB/s eta 0:00:04
   ---- ----------------------------------- 1.2/10.7 MB 2.7 MB/s eta 0:00:04
   ---- ------

In [38]:
import requests
from requests_html import HTML
import time
import pandas as pd




In [39]:
base_url = "https://stackoverflow.com/questions/tagged/"
tag = "python"
query_filter = "Votes"
url = f"{base_url}{tag}>tab={query_filter}"
url

'https://stackoverflow.com/questions/tagged/python>tab=Votes'

In [40]:
print(url)

https://stackoverflow.com/questions/tagged/python>tab=Votes


In [20]:
#from requests_html import HTML

In [21]:
#html = HTML(html=html_str)

In [19]:
question_summaries = html.find(".s-post-summary")
print(question_summaries)

NameError: name 'html' is not defined

In [193]:
question_summaries[0].text

"0 votes\n0 answers\n4 views\nScraping form elements in selenium python without specifying class/ID?\nFor other pages I specified the elements, but this pages field is dynamic based on the zip. Is there a way to scrape the values, in the form since they will change? I don't want to have to specify for ...\npython\nselenium-webdriver\nbeautifulsoup\nCodyjane\n1\nasked 2 mins ago"

In [194]:
question_summaries[1].text

"0 votes\n0 answers\n4 views\nFast evaluation of function of multiple derivatives in sympy\nI have a generic symbolic function x(t) [think of t as time], and a function of various derivatives of x, ie F(x(t),x'(t),x''(t),..., x^(n)(t)). I would like to evaluate F as fast as possible given ...\npython\nsympy\nLance\n125\nasked 2 mins ago"

In [23]:
columns = ['votes', 'answers', 'views', 'short_description', 'question', 'tag1','tag2', 'tag3','user', 'user-details',  'date']

In [24]:
this_row = list(question_summaries[0].text.split("\n"))
this_row

NameError: name 'question_summaries' is not defined

In [78]:
#len(this_row) == len(columns)

True

In [79]:
#row_data = dict(zip(columns, this_row))
#row_data

{'votes': '0 votes',
 'answers': '0 answers',
 'views': '4 views',
 'short_description': 'Scraping form elements in selenium python without specifying class/ID?',
 'question': "For other pages I specified the elements, but this pages field is dynamic based on the zip. Is there a way to scrape the values, in the form since they will change? I don't want to have to specify for ...",
 'tag1': 'python',
 'tag2': 'selenium-webdriver',
 'tag3': 'beautifulsoup',
 'user': 'Codyjane',
 'user-details': '1',
 'date': 'asked 2 mins ago'}

In [81]:
#for column, row_v in zip(columns, this_row):
   # print(column, row_v)

votes 0 votes
answers 0 answers
views 4 views
short_description Scraping form elements in selenium python without specifying class/ID?
question For other pages I specified the elements, but this pages field is dynamic based on the zip. Is there a way to scrape the values, in the form since they will change? I don't want to have to specify for ...
tag1 python
tag2 selenium-webdriver
tag3 beautifulsoup
user Codyjane
user-details 1
date asked 2 mins ago


In [20]:
key_names = ['question', 'metrics', 'tags']
classes_needed = ['.s-link', '.s-post-summary--stats', '.post-tag']
this_question_element = question_summaries[0]
this_question_element.find('.s-link', first=True).text
this_question_element.find('.s-post-summary--stats', first=True).text.replace('\n', ' ')


NameError: name 'question_summaries' is not defined

In [26]:
def clean_scraped_data(text, keyname=None):
    if keyname == 'metrics':
        return text.replace('\n', ' ')
    #if keyname == 'post-tag':
        #return text.replace('\n', ',')
    return text

In [27]:
datas = []

for q_el in question_summaries:
    question_data = {}
    for i, _class in enumerate(classes_needed):
        sub_el = q_el.find(_class, first=True)
        keyname = key_names[i]
        question_data[keyname] = clean_scraped_data(sub_el.text, keyname=keyname)
    datas.append(question_data)



NameError: name 'question_summaries' is not defined

In [28]:
def parse_tagged_page(html):
    question_summaries = html.find(".s-post-summary")
    key_names = ['question', 'metrics', 'tags']
    classes_needed = ['.s-link', '.s-post-summary--stats', '.post-tag']
    datas = []
    for q_el in question_summaries:
        question_data = {}
        for i, _class in enumerate(classes_needed):
            sub_el = q_el.find(_class, first=True)
            keyname = key_names[i]
            question_data[keyname] = clean_scraped_data(sub_el.text, keyname=keyname)
        datas.append(question_data)
    return datas



In [29]:
def extract_data_from_url(url):
    r = requests.get(url)
    if r.status_code not in range(200, 299):
        return[]
    html_str = r.text
    html = HTML(html=html_str)
    datas = parse_tagged_page(html)
    return datas
    

In [30]:
def scrape_tag(tag = "python", query_filter = "Votes", max_pages=50, pagesize=50):
    base_url = "https://stackoverflow.com/questions/tagged/"
    datas = []
    for p in range(max_pages):
        page_num = p + 1
        url = f"{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={pagesize}"
        datas += extract_data_from_url(url)
        time.sleep(1.2)
    return datas
        

In [31]:
datas = scrape_tag(tag="python")
print(datas)


[{'question': 'What does the "yield" keyword do in Python?', 'metrics': '12765 votes 50 answers 3.3m views', 'tags': 'python'}, {'question': 'What does if __name__ == "__main__": do?', 'metrics': '8106 votes 46 answers 4.5m views', 'tags': 'python'}, {'question': 'Does Python have a ternary conditional operator?', 'metrics': '7824 votes 32 answers 2.8m views', 'tags': 'python'}, {'question': 'What are metaclasses in Python?', 'metrics': '7307 votes 25 answers 1.1m views', 'tags': 'python'}, {'question': 'How do I check whether a file exists without exceptions?', 'metrics': '7055 votes 40 answers 5.3m views', 'tags': 'python'}, {'question': 'How do I merge two dictionaries in a single expression in Python?', 'metrics': '6838 votes 43 answers 3.2m views', 'tags': 'python'}, {'question': 'How do I execute a program or call a system command?', 'metrics': '6051 votes 66 answers 4.5m views', 'tags': 'python'}, {'question': 'How do I create a directory, and any missing parent directories?', '

In [32]:
len(datas)

2500

In [33]:
df = pd.DataFrame(datas)
df.head()

,question,metrics,tags
0,"What does the ""yield"" keyword do in Python?",12765 votes 50 answers 3.3m views,python
1,"What does if __name__ == ""__main__"": do?",8106 votes 46 answers 4.5m views,python
2,Does Python have a ternary conditional operator?,7824 votes 32 answers 2.8m views,python
3,What are metaclasses in Python?,7307 votes 25 answers 1.1m views,python
4,How do I check whether a file exists without e...,7055 votes 40 answers 5.3m views,python


In [34]:
df.shape

(2500, 3)

In [35]:
#df.to_csv("Python.csv", index=False)

In [ ]:
# could consider adding machine learning with   auto-tagging question bot

In [44]:
def scrape_link(tag = "python", query_filter = "Votes", id = id, max_pages=50, pagesize=50):
    base_url = "https://stackoverflow.com/questions/{id}"
    datas = []
    for p in range(max_pages):
        page_num = p + 1
        url = f"{base_url}{id}?tab={query_filter}&page={page_num}&pagesize={pagesize}"
        datas += extract_data_from_url(url)
        time.sleep(1.2)
    return datas